# Capptu Categories Approach to createMl model

     ## Results from createml 
     
     
     
    ******CONFUSION MATRIX******
    ----------------------------------
    True\Pred                    Animales_y_mascotas Arquitectura_y_Edificios Comida_y_Bebidas Deportes_y_Acondicionamiento  Naturaleza  Transportes                   
    Animales_y_mascotas          1615                41                       53               15                            191         13                            
    Arquitectura_y_Edificios     27                  1543                     49               60                            192         82                            
    Comida_y_Bebidas             36                  55                       1697             28                            62          19                            
    Deportes_y_Acondicionamiento 101                 340                      158              775                           340         171                           
    Naturaleza                   125                 169                      60               71                            1485        51                            
    Transportes                  44                  286                      60               158                           226         1135                          

    ******PRECISION RECALL******
    ----------------------------------
    Class                        Precision(%)   Recall(%)      
    Animales_y_mascotas          82.91          83.77          
    Arquitectura_y_Edificios     63.39          79.01          
    Comida_y_Bebidas             81.70          89.46          
    Deportes_y_Acondicionamiento 70.01          41.11          
    Naturaleza                   59.50          75.73          
    Transportes                  77.16          59.46          



    ----------------------------------
    Number of examples: 11533
    Number of classes: 6
    Accuracy: 71.53%





In [ ]:
import os 
import sys
import keras
from PIL import  Image
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D
from keras.models import  Model, Sequential
from keras.optimizers import Adam
from __future__ import print_function
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
image_dimensions = (128, 128)

In [ ]:
"""Defining image generators"""

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, images_path, batch_size_per_class=32, dim=(256,256), n_classes=6, data_augm=False):
        category_images = dict()
        categories = os.listdir(images_path)
        mean_image = cv2.resize(cv2.imread('./mean_image.jpg'), dim)
        clases = {'Naturaleza':0,
         'Deportes_y_Acondicionamiento':1,
         'Transportes':2,
         'Animales_y_mascotas':3,
         'Arquitectura_y_Edificios':4,
         'Comida_y_Bebidas':5}
        for x in categories:
            images = os.listdir( images_path + x ) 
            category_images [x]  = images
        self.data_augm = data_augm
        self.clases =  clases
        self.images_path = images_path
        self.dim = dim
        self.batch_size = batch_size_per_class
        self.n_classes = n_classes
        self.category_images = category_images
        self.mean_image = mean_image
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return 10
    

    def __getitem__(self, index):
        chosen = dict()
        for category in self.category_images.keys():
            random.shuffle(self.category_images[category])
            chosen [category] = self.category_images[category][: self.batch_size]
        # Generate data
        X, y = self.__data_generation(chosen)
        return np.array(X), to_categorical(y)
    
    def on_epoch_end(self):
        """Nothing"""

    def __data_generation(self, chosen):
        x = []
        y = []
        angles = [0,90,180,270]
        categories = chosen.keys()
        for category in chosen.keys():
            for image_name in chosen[category]:
#                 im = Image.open( self.images_path +category+'/'+image_name)
                image = cv2.resize(cv2.imread(self.images_path +category + '/' + image_name), self.dim) - self.mean_image
                if self.data_augm is True:
                    (h,w,ch) = image.shape
                    center = (w/2, h/2)
                    r = np.random.randint(0,4)
                    f = np.random.randint(0,2)
                    M=cv2.getRotationMatrix2D(center, angles[r], 1)
                    image = cv2.warpAffine(image,M,(h,w))
                    image = cv2.flip(image,f)
                    if random.random() > .5:
                        kernel = np.ones((5,5),np.float32)/25
                        image = cv2.filter2D(image,-1,kernel)
                x.append(image)
                y.append(self.clases[category])
        return x, y
# train_generator = DataGenerator(
#                     images_path='./train/',
#                     batch_size_per_class=6,
#                     data_augm=True, 
#                     dim=image_dimensions) # 6 classes -> 36 photos per batch
# test_generator = DataGenerator(
#                     images_path='./test/',
#                     batch_size_per_class=4, 
#                     data_augm=False,
#                     dim=image_dimensions)   

train_datagen = ImageDataGenerator(
        featurewise_center=True,
        samplewise_center=True,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range= 45,
        channel_shift_range=.5,
        fill_mode='constant'
)

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=image_dimensions,
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './test/',
        target_size=image_dimensions,
        batch_size=32,
        class_mode='categorical')


In [ ]:
x,y = train_generator.__getitem__(1)
for i in range(len(x)):
    plt.imshow(x[i])
    plt.show()

In [ ]:
# create model
def m_2():
    model = Sequential()#add model layers
    model.add(InputLayer(input_shape=(244, 244,3)))
    model.add(Conv2D(8, kernel_size=8, activation='relu'))
    model.add(Conv2D(16, kernel_size=8, activation='relu'))
    model.add(Flatten())
    model.add(Dense(6, activation='softmax'))
    adam = Adam(lr=0.001, decay=0.00000001, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= ['mae'])
    model.summary()

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(244,244,3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    model.summary()
    return model

def m_1():
    model = Sequential()
    model.add(Conv2D(32, 
                     kernel_size=(3, 3),
                     input_shape=(image_dimensions[0], 
                                  image_dimensions[1], 3),
                     activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6), activation='softmax')
    model.add(Activation('sigmoid'))

    model.compile(loss=binary_crossentropy,
                  optimizer=RMSprop(lr=0.0005),  # half of the default lr
                  metrics=['accuracy'])
    model.summary()
    return model

def m_0(core_model_trainable_layers = 2):
    inp = Input(shape=(image_dimensions[0], image_dimensions[1], 3))
    x = MobileNetV2(weights='imagenet', include_top=True)(inp)
    x = Dropout(.5)(x)
    x = Dense(100, activation='softmax')(x)
    x = Dropout(.5)(x)
    x = Dense(6, activation='softmax')(x)
    
    model = Model(inp, x)
    adam = Adam(lr=0.001, decay=0.00000001, amsgrad=False)
    l_lenght = len(model.layers[1].layers)
    for i, layer in enumerate(model.layers[1].layers):#[:-core_model_trainable_layers]:
        if i < l_lenght - core_model_trainable_layers:
            layer.trainable = False
        else:
            layer.trainable = True
    model.compile(loss='mean_squared_error', optimizer='adam', metrics= ['mae', 'acc'])
    model.summary()
    return model

In [ ]:
model = m_0()

In [ ]:
history = model.fit_generator(
        generator=train_generator, 
        validation_data=test_generator, 
        steps_per_epoch=1588, 
        validation_steps=400, 
        epochs=100)

In [ ]:
plt.plot(model.history.history['val_acc'])

    # to resize images
    # import os                                            
    # from PIL import Image                                     
    # import sys                                      
    # def resize():
    #     origin_path = './filtered_categories/'                               
    #     destination_path = './filtered_categories_resized/'
    #     cats = os.listdir(origin_path)
    #     for cat in cats:
    #         os.system('rm -rv ' + destination_path + cat)
    #         os.system('mkdir ' + destination_path + cat)

    #     for cat in cats:
    #         print cat
    #         for im_name in os.listdir(origin_path + cat):
    #             try:            
    #                 im = Image.open(origin_path + cat + '/' + im_name).resize((512,512))
    #                 im.save(destination_path + cat + '/' + im_name)                                                                                  
    #                 print cat, im_name
    #             except:
    #                 print "                       Error" + im_name
    # resize()
    
    
    # import os                                            
    # import sys              
    # import random
    # def move():
    #     origin_path = './filtered_categories_resized/'                               
    #     destination_path = './filtered_categories_resized_test/'
    #     cats = os.listdir(origin_path)

    #     for cat in cats:
    #         os.system('mkdir ' + destination_path + cat)

    #     for cat in cats:
    #         print cat
    #         files = os.listdir(origin_path + cat)

    #         files_to_move = int(.2 * len(files))

    #         random.shuffle(files)
    #         for im_name in files[:files_to_move]:
    #             os.system('mv ' + origin_path + cat + '/' + im_name + ' '+destination_path+ cat+ '/' + im_name )

    # move()
    
    
    
    
    # Partition Data 
    # import os                                            
    # import sys              
    # import random
    # def move():
    #     origin_path = './filtered_categories_resized/'                               
    #     destination_path = './filtered_categories_resized_test/'
    #     cats = os.listdir(origin_path)

    #     for cat in cats:
    #         os.system('mkdir ' + destination_path + cat)

    #     for cat in cats:
    #         print cat
    #         files = os.listdir(origin_path + cat)

    #         files_to_move = int(.2 * len(files))

    #         random.shuffle(files)
    #         for im_name in files[:files_to_move]:
    #             os.system('mv ' + origin_path + cat + '/' + im_name + ' '+destination_path+ cat+ '/' + im_name )